## Compress DICOMs using JPEG 2000 Lossless compression
- take a large 1.9GB DICOM multi-frame image and compress each frame

In [0]:
%pip install --quiet numpy==1.26.4 pydicom  nvidia-nvimgcodec-cu12 nvidia-nvjpeg2k-cu12

In [0]:
dbutils.library.restartPython()

In [0]:
import pydicom
import numpy as np
from nvidia import nvimgcodec
from pydicom.encaps import encapsulate
from pydicom.uid import JPEG2000Lossless  # or JPEG2000 for lossy


# Path to your input DICOM file (uncompressed)
input_path = "/Volumes/hls_radiology/demo/random/20250227134948_CT_ISRA_0.dcm"

# Output path for the compressed DICOM file
output_path = "/Volumes/hls_radiology/demo/random/20250227134948_CT_ISRA_0-jpeg2000-nv-compressed.dcm"


In [0]:
encoded_frames = []
encoder = nvimgcodec.Encoder()
#params = nvimgcodec.Jpeg2kEncodeParams(reversible=True)
params = nvimgcodec.Jpeg2kEncodeParams(ht=True)
encode_params = nvimgcodec.EncodeParams(jpeg2k_encode_params=params)

In [0]:

# Read the multi-frame DICOM
ds = pydicom.dcmread(input_path)
pixel_array = ds.pixel_array  # shape: (frames, rows, cols)



In [0]:
# for each frame
for i in range(pixel_array.shape[0]):
    frame = pixel_array[i]
    if frame.ndim == 2:
        frame = frame[:, :, np.newaxis]  # Shape: (rows, cols, 1)
    frame = np.ascontiguousarray(frame)
    nv_img = nvimgcodec.as_image(frame.astype(np.uint16))  # Adjust dtype if needed
    j2k_bytes = encoder.encode(nv_img, 'jpeg2k', params=encode_params)
    encoded_frames.append(j2k_bytes)


In [0]:
# Encapsulate all JPEG 2000 encoded frames
ds.PixelData = encapsulate(encoded_frames)
ds['PixelData'].is_undefined_length = True

In [0]:
ds.file_meta.TransferSyntaxUID = JPEG2000Lossless
ds.save_as(output_path)

In [0]:
%sh ls -laht \
/Volumes/hls_radiology/demo/random/20250227134948_CT_ISRA_0*

In [0]:
%sh file /Volumes/hls_radiology/demo/random/20250227134948_CT_ISRA_0*